<a href="https://colab.research.google.com/github/AdamKimhub/Msproject1/blob/colab/forfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    dataset_path = '/content/drive/My Drive/dataset'

    # Install required packages
    !pip install pymatgen torch torch_geometric

else:
    dataset_path = 'dataset'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 M

In [3]:
from pathlib import Path
import ast
import numpy as np
import pandas as pd
from pymatgen.core import Structure, PeriodicSite, DummySpecie
from pymatgen.analysis.local_env import MinimumDistanceNN


## When  dealing with high dataset only

In [ ]:
high_dataset = ["high_BN", "high_GaSe", "high_InSe", "high_MoS2", "high_P", "high_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{high_data}.csv") for high_data in high_dataset]

high_df  = pd.concat(to_merge, ignore_index=True)

high_copy = high_df.copy()

In [ ]:
high_copy = high_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
high_copy = high_copy.corr()
high_copy

## When dealing with low dataset

In [ ]:
low_dataset = ["low_MoS2", "low_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{low_data}.csv") for low_data in low_dataset]

low_df  = pd.concat(to_merge, ignore_index=True)

low_copy = low_df.copy()

In [ ]:
low_copy = low_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
low_copy = low_copy.corr()
low_copy

## When dealing with both low and high dataset

In [ ]:
# Read file
sample_df = pd.read_csv(f"{dataset_path}/combined/combined.csv")

In [ ]:
sample_copy = sample_df.copy()
sample_copy = sample_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
sample_copy = sample_copy.corr()
sample_copy

## Data to graphs

In [10]:
samplex_df = pd.read_csv(f"{dataset_path}/combined/high_WSe2.csv")

In [54]:
dat = "high_WSe2"
bas = "WSe2"
cel = [8,8,1]

# Get number of site
the_cif = f"{dataset_path}/{dat}/{bas}.cif"
ref_cell = Structure.from_file(the_cif)
ref_struct = ref_cell.make_supercell(cel)

num_site = ref_struct.num_sites
num_site

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


192

In [55]:
(samplex_df["defect_sites"][0]/num_site)*100

np.float64(2.083333333333333)

In [8]:
import to_graph
import torch
from torch_geometric.data import Data

In [11]:

def graphy(row):
    defective_file_path = Path(f"{dataset_path}/{row['dataset_material']}/cifs/{row['_id']}.cif")
    defective_structure = Structure.from_file(defective_file_path)

    ref_file_path = Path(f"{dataset_path}/{row['dataset_material']}/{row['base']}.cif")
    ref_unit_cell = Structure.from_file(ref_file_path)
    the_cell = ast.literal_eval(row['cell'])
    reference_structure = ref_unit_cell.make_supercell(the_cell)

    defects_structure = to_graph.get_defects_structure(defective_structure, reference_structure)

    the_nodes, the_edges, the_edge_features = to_graph.get_nodes_edges(defects_structure)

    global_attributes = ["energy","fermi_level","total_mag","formation_energy",
                         "energy_per_atom","E_1","vacancy_sites",
                         "substitution_sites","defect_sites"]

    global_features = [row[i] for i in global_attributes]

    target_attribute = "band_gap_value"
    target_features = [row[target_attribute]]

    the_data = Data(x=torch.tensor(the_nodes), edge_index=torch.tensor(the_edges),
                    edge_attr=torch.tensor(the_edge_features),
                    u=torch.tensor(global_features), y=torch.tensor(target_features))
    return the_data

samplex = samplex_df.apply(lambda row: graphy(row), axis = 1).tolist()

# Save the data
torch.save(samplex, f"{dataset_path}/combined/all_graphs.pt")

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 48 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 47 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 46 fractional coordinates rounded to ideal values to avoid is

## Build model with the graph data

In [12]:
# Load the data
whole_graph = torch.load(f"{dataset_path}/combined/all_graphs.pt", weights_only=False)

In [58]:
whole_graph[499].u

tensor([-1281.8348,    -1.8732,     0.0000,    42.7350,    -7.1213,   -63.3783,
           12.0000,    12.0000,    24.0000])

In [56]:
# Split the data
from sklearn.model_selection import train_test_split

train_graphs, test_graphs = train_test_split(whole_graph, test_size=0.3, random_state=42)

val_graphs, test_graphs = train_test_split(test_graphs, test_size=0.5, random_state=42)

In [57]:
# Create data loaders
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)
val_loader = DataLoader(val_graphs, batch_size=32, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

# 1. Define GNN Model
class GraphLevelGNN(nn.Module):
    def __init__(self, node_dim, global_dim, hidden_dim):
        super().__init__()
        self.conv1 = GCNConv(node_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim + global_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)  # For regression

    def forward(self, x, edge_index, batch, global_attr):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Aggregate node info to graph-level
        x = torch.cat([x, global_attr], dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

'''# 2. Create synthetic dataset
def generate_synthetic_graph():
    num_nodes = 5
    x = torch.randn((num_nodes, 3))  # node_dim = 3

    # Fully connected graph (no self-loops)
    edge_index = torch.combinations(torch.arange(num_nodes), r=2).T
    edge_index = torch.cat([edge_index, edge_index[[1, 0]]], dim=1)  # Make undirected

    global_feat = torch.randn((1, 2))  # global_dim = 2
    target = torch.tensor([[random.uniform(0, 1)]])  # some random scalar target

    data = Data(x=x, edge_index=edge_index, u=global_feat, y=target)
    return data
'''
# Instantiate model, optimizer, loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphLevelGNN(node_dim=7, global_dim=9, hidden_dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()


# Training loop
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch, data.u)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

# Evaluation (val/test)
def evaluate(loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch, data.u)
            loss = loss_fn(out, data.y)
            total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

# Run training and testing
for epoch in range(1, 31):
    train_loss = train()
    val_loss = evaluate(val_loader)
    print(f'Epoch {epoch:03d}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

test_loss = evaluate(test_loader)
print(f'Test Loss: {test_loss:.4f}')


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 182 but got size 12 for tensor number 2 in the list.

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import NNConv, global_mean_pool
import random

# Model with edge features
class GNNWithEdgeFeatures(nn.Module):
    def __init__(self, node_dim, edge_dim, global_dim, hidden_dim):
        super().__init__()
        # Edge NN maps edge_attr to weight matrix
        self.edge_nn = nn.Sequential(
            nn.Linear(edge_dim, 32),
            nn.ReLU(),
            nn.Linear(32, node_dim * hidden_dim)
        )

        self.conv1 = NNConv(node_dim, hidden_dim, self.edge_nn, aggr='mean')
        self.conv2 = NNConv(hidden_dim, hidden_dim, self.edge_nn, aggr='mean')

        self.fc1 = nn.Linear(hidden_dim + global_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)  # Regression output

    def forward(self, x, edge_index, edge_attr, batch, global_attr):
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x = global_mean_pool(x, batch)
        x = torch.cat([x, global_attr], dim=1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Generate synthetic graphs
def generate_synthetic_graph():
    num_nodes = 5
    x = torch.randn((num_nodes, 3))  # node_dim = 3

    # Fully connected graph (no self-loops)
    edge_index = torch.combinations(torch.arange(num_nodes), r=2).T
    edge_index = torch.cat([edge_index, edge_index[[1, 0]]], dim=1)  # Make undirected

    # Edge features = Euclidean distance between connected nodes
    edge_attr = []
    for i, j in edge_index.t():
        dist = torch.norm(x[i] - x[j]).unsqueeze(0)  # shape [1]
        edge_attr.append(dist)
    edge_attr = torch.stack(edge_attr)  # shape [num_edges, 1]

    u = torch.randn((1, 2))  # global features
    y = torch.tensor([[random.uniform(0, 1)]])  # target

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, u=u, y=y)
    return data

# Create dataset
dataset = [generate_synthetic_graph() for _ in range(300)]
train_data = dataset[:200]
val_data = dataset[200:250]
test_data = dataset[250:]

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)
test_loader = DataLoader(test_data, batch_size=16)

# Setup model, loss, optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNWithEdgeFeatures(node_dim=3, edge_dim=1, global_dim=2, hidden_dim=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Train function
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr, data.batch, data.u)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

# Evaluation function
def evaluate(loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.edge_attr, data.batch, data.u)
            loss = loss_fn(out, data.y)
            total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

# Run training
for epoch in range(1, 31):
    train_loss = train()
    val_loss = evaluate(val_loader)
    print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

# Final test
test_loss = evaluate(test_loader)
print(f"\nTest Loss: {test_loss:.4f}")


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


RuntimeError: The size of tensor a (320) must match the size of tensor b (15) at non-singleton dimension 0

In [ ]:
from torch_geometric.nn import GCNConv, global_mean_pool

class GNNGraphLevelModel(torch.nn.Module):
    def __init__(self, node_dim, global_dim, hidden_dim):
        super().__init__()
        self.conv1 = GCNConv(node_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc1 = torch.nn.Linear(hidden_dim + global_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)  # For regression

    def forward(self, x, edge_index, batch, global_attr):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = global_mean_pool(x, batch)  # shape [batch_size, hidden_dim]

        x = torch.cat([x, global_attr], dim=1)  # combine with global features
        x = self.fc1(x).relu()
        return self.fc2(x)

In [ ]:
import torch.nn as nn

class DefectsModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DefectsModel, self).__init__()
